In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from keras.optimizers import SGD, Adam, RMSprop, Adadelta, Adagrad
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import math
from sklearn.metrics import mean_squared_error

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
dataset = pd.read_csv('/content/drive/MyDrive/Marek/Uni/Seminar_25/time-series_data.csv')

In [5]:
dataset.dropna(inplace=True)

In [6]:
#Spalte 'week' in datetime konvertieren
dataset['week'] = pd.to_datetime(dataset['week'], format='%d/%m/%y')

In [7]:
# Vorbereitung der Variablen

numeric_features = ['units_sold', 'total_price', 'base_price']
categorical_features = ['store_id', 'sku_id']

# Label Encoding für kategoriale Features
le_store = LabelEncoder()
le_sku = LabelEncoder()

dataset['store_id_encoded'] = le_store.fit_transform(dataset['store_id'])
dataset['sku_id_encoded'] = le_sku.fit_transform(dataset['sku_id'])

# Neue Features für die Sequenzierung, inklusive der binären Variablen
features = numeric_features + ['is_featured_sku', 'is_display_sku', 'store_id_encoded', 'sku_id_encoded']

In [8]:
# Erstellung Trainingsset

# Schritt 1: Eindeutige 'week'-Werte extrahieren
unique_weeks = dataset.index.unique()
n_weeks = len(unique_weeks)

weeks_train = int(0.75 * n_weeks)

# Schritt 2: Definiere die Grenzen für die Sets
train_weeks = unique_weeks[:weeks_train]
rest_weeks = unique_weeks[weeks_train:]

# Schritt 3: Daten in die jeweiligen Sets aufteilen
train_set = dataset[dataset.index.isin(train_weeks)]
print("Trainingsset:")
print(f"Anzahl der Reihen: {train_set.shape[0]}, mögliche Länge der Sequenzen: {train_set.index.nunique()}")

rest_set = dataset[dataset.index.isin(rest_weeks)]
print("Rest-Set:")
print(f"Anzahl der Reihen: {rest_set.shape[0]}, mögliche Länge der Sequenzen: {rest_set.index.nunique()}")

Trainingsset:
Anzahl der Reihen: 112611, mögliche Länge der Sequenzen: 112611
Rest-Set:
Anzahl der Reihen: 37538, mögliche Länge der Sequenzen: 37538


In [ ]:
# Erstellung Validierungs- und Testset

# Schritt 1: Gruppierung nach 'store_id' und 'sku_id'
rest_grouped = list(rest_set.groupby(['store_id', 'sku_id']))

# Schritt 2: Shuffle der Gruppen, um eine zufällige Verteilung zu gewährleisten
np.random.seed(42)  # für Reproduzierbarkeit
np.random.shuffle(rest_grouped)

# Schritt 3: Aufteilung der Gruppen in Sets
n_groups = len(rest_grouped)
split = int(0.5 * n_groups)

val_groups = rest_grouped[:split]
test_groups = rest_grouped[split:]

# Schritt 4: Neue DataFrames für die Sets erstellen
val_set = pd.concat([group for _, group in val_groups])
test_set = pd.concat([group for _, group in test_groups])

# Überprüfung der Größen
print("Validierungsset:")
print(f"Anzahl der Reihen: {val_set.shape[0]}, Anzahl der Gruppen: {len(val_groups)}, Länge der Reihen: {val_set.index.nunique()}")
print("Test-Set:")
print(f"Anzahl der Reihen: {test_set.shape[0]}, Anzahl der Gruppen: {len(test_groups)}, Länge der Reihen: {test_set.index.nunique()}")

Validierungsset:
Anzahl der Reihen: 18739, Anzahl der Gruppen: 577, Länge der Reihen: 18739
Test-Set:
Anzahl der Reihen: 18799, Anzahl der Gruppen: 578, Länge der Reihen: 18799


In [ ]:
# Index zurücksetzen

# Schritt 1: Den Index wieder auf den datetime 'week' setzen (falls nicht mehr so)
train_set = train_set.set_index('week')
val_set = val_set.set_index('week')
test_set = test_set.set_index('week')

# Schritt 2: Nach 'week' sortieren
train_set = train_set.sort_index()
val_set = val_set.sort_index()
test_set = test_set.sort_index()

In [ ]:
print("Trainingsset:")
print(f"Anzahl der Reihen: {train_set.shape[0]}, mögliche Länge der Sequenzen: {train_set.index.nunique()}")
print()
print("Validierungsset:")
print(f"Anzahl der Reihen: {val_set.shape[0]}, Anzahl der Gruppen: {len(val_groups)}, Länge der Reihen: {val_set.index.nunique()}")
print()
print("Test-Set:")
print(f"Anzahl der Reihen: {test_set.shape[0]}, Anzahl der Gruppen: {len(test_groups)}, Länge der Reihen: {test_set.index.nunique()}")

Trainingsset:
Anzahl der Reihen: 112611, mögliche Länge der Sequenzen: 98

Validierungsset:
Anzahl der Reihen: 18739, Anzahl der Gruppen: 577, Länge der Reihen: 33

Test-Set:
Anzahl der Reihen: 18799, Anzahl der Gruppen: 578, Länge der Reihen: 33


In [ ]:
print("Trainingsset:")
train_set.info()
train_set.head()

Trainingsset:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 112611 entries, 2011-01-17 to 2012-11-27
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   record_ID         112611 non-null  int64  
 1   store_id          112611 non-null  int64  
 2   sku_id            112611 non-null  int64  
 3   total_price       112611 non-null  float64
 4   base_price        112611 non-null  float64
 5   is_featured_sku   112611 non-null  int64  
 6   is_display_sku    112611 non-null  int64  
 7   units_sold        112611 non-null  int64  
 8   store_id_encoded  112611 non-null  int64  
 9   sku_id_encoded    112611 non-null  int64  
dtypes: float64(2), int64(8)
memory usage: 13.5 MB


,record_ID,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold,store_id_encoded,sku_id_encoded
week,,,,,,,,,,
2011-01-17,1,8091,216418,99.0375,111.8625,0,0,20,3,1
2011-01-17,2,8091,216419,99.0375,99.0375,0,0,28,3,2
2011-01-17,3,8091,216425,133.9500,133.9500,0,0,19,3,3
2011-01-17,4,8091,216233,133.9500,133.9500,0,0,44,3,0
2011-01-17,5,8091,217390,141.0750,141.0750,0,0,52,3,5


In [ ]:
print("Validierungsset:")
val_set.info()
val_set.head()

Validierungsset:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 18739 entries, 2012-11-27 to 2013-07-09
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   record_ID         18739 non-null  int64  
 1   store_id          18739 non-null  int64  
 2   sku_id            18739 non-null  int64  
 3   total_price       18739 non-null  float64
 4   base_price        18739 non-null  float64
 5   is_featured_sku   18739 non-null  int64  
 6   is_display_sku    18739 non-null  int64  
 7   units_sold        18739 non-null  int64  
 8   store_id_encoded  18739 non-null  int64  
 9   sku_id_encoded    18739 non-null  int64  
dtypes: float64(2), int64(8)
memory usage: 2.1 MB


,record_ID,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold,store_id_encoded,sku_id_encoded
week,,,,,,,,,,
2012-11-27,160008,9823,673209,355.5375,355.5375,0,0,49,63,26
2012-11-27,159508,9439,219009,190.2375,190.2375,0,0,60,41,7
2012-11-27,159667,9498,216419,86.9250,86.9250,0,0,34,47,2
2012-11-27,160007,9823,547934,177.4125,177.4125,0,0,23,63,24
2012-11-27,160288,9984,245338,391.8750,469.5375,1,0,23,75,14


In [ ]:
print("Testset:")
test_set.info()
test_set.head()

Testset:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 18799 entries, 2012-11-27 to 2013-07-09
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   record_ID         18799 non-null  int64  
 1   store_id          18799 non-null  int64  
 2   sku_id            18799 non-null  int64  
 3   total_price       18799 non-null  float64
 4   base_price        18799 non-null  float64
 5   is_featured_sku   18799 non-null  int64  
 6   is_display_sku    18799 non-null  int64  
 7   units_sold        18799 non-null  int64  
 8   store_id_encoded  18799 non-null  int64  
 9   sku_id_encoded    18799 non-null  int64  
dtypes: float64(2), int64(8)
memory usage: 2.1 MB


,record_ID,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold,store_id_encoded,sku_id_encoded
week,,,,,,,,,,
2012-11-27,160238,9954,245338,391.8750,469.5375,1,0,27,73,14
2012-11-27,160002,9823,223153,236.5500,236.5500,0,0,40,63,12
2012-11-27,160039,9809,219029,327.0375,327.0375,0,0,36,61,8
2012-11-27,159614,9479,222765,175.9875,227.2875,0,0,50,44,11
2012-11-27,159616,9479,223153,235.8375,235.8375,0,0,26,44,12


In [ ]:
# Skalierer nur für die numerischen Features (ohne binäre)
scalers = {feature: MinMaxScaler() for feature in numeric_features}

for feature in numeric_features:
  scalers[feature].fit(train_set[feature].values.reshape(-1, 1))

In [ ]:
# Funktion zur Erstellung von Sequenzen
def create_sequences(dataset, features, numeric_features, scalers, timesteps):
    X_all = []
    y_all = []

    grouped = dataset.groupby(['store_id', 'sku_id'])

    for (store_id, sku_id), group in grouped:
        group = group.sort_index()

        # Daten extrahieren
        data = group[features].values

        # Skalieren der numerischen Features
        for feature in numeric_features:
            index = features.index(feature)
            data[:, index] = scalers[feature].transform(data[:, index].reshape(-1, 1)).flatten()

        # Kategoriale Features bleiben als Integer

        for i in range(timesteps, len(data)):
            X_seq = data[i - timesteps:i]
            X_all.append(X_seq)
            y_all.append(data[i, 0])  # units_sold ist der erste Wert in features

    X_array = np.array(X_all)
    y_array = np.array(y_all)
    return X_array, y_array

In [ ]:
# Liste der Fenstergrößen
window_sizes = [30]

# Dictionary, um die erzeugten Datensätze zu speichern
train_sets = {}
val_sets = {}

for window in window_sizes:
    X_train, y_train = create_sequences(train_set, features, numeric_features, scalers, window)
    train_sets[window] = {'X': X_train, 'y': y_train}

    X_val, y_val = create_sequences(val_set, features, numeric_features, scalers, window)
    val_sets[window] = {'X': X_val, 'y': y_val}

    print(f"Fenstergröße {window}:")
    print(f"Trainings-Sequenzen: {X_train.shape}, Validierungs-Sequenzen: {X_val.shape}.")

Fenstergröße 30:
Trainings-Sequenzen: (77961, 30, 7), Validierungs-Sequenzen: (1429, 30, 7).


In [ ]:
X_tr = train_sets[30]['X']
y_tr = train_sets[30]['y']
X_v = val_sets[30]['X']
y_v = val_sets[30]['y']

In [ ]:
dropout_rates = [0.2, 0.5]

for dropout in dropout_rates:
    print(f"Training mit Dropout={dropout}")

    model = Sequential()
    # Erste Schicht
    model.add(LSTM(units=64,
                    input_shape=(X_tr.shape[1], X_tr.shape[2]),
                    dropout=dropout,
                    return_sequences=True))
    # Zweite Schicht
    model.add(LSTM(units=64, dropout=dropout))

    model.add(Dense(1, activation='linear'))

    # Kompilieren
    model.compile(optimizer=SGD(learning_rate=0.01),
                  loss='mean_squared_error')

    # Training mit Verlaufsdaten sammeln
    history = model.fit(X_tr, y_tr, epochs=10, batch_size=64,
                              validation_data=(X_v, y_v))

    # Verlaufsdaten speichern
    train_losses = history.history['loss']
    val_losses = history.history['val_loss']

    # Ausgabe pro Modell
    avg_val_loss = np.mean(val_losses)
    best_val_loss = np.min(val_losses)
    print(f"Durchschnittlicher Validierungsverlust: {avg_val_loss:.6f}")
    print(f"Bester Validierungsverlust: {best_val_loss:.6f}")

Training mit Dropout=0.2
Epoch 1/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 55s 44ms/step - loss: 0.0061 - val_loss: 5.2205e-04
Epoch 2/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 82s 44ms/step - loss: 0.0014 - val_loss: 5.4131e-04
Epoch 3/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 84s 46ms/step - loss: 0.0011 - val_loss: 4.9737e-04
Epoch 4/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 81s 45ms/step - loss: 8.9092e-04 - val_loss: 4.9523e-04
Epoch 5/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 82s 45ms/step - loss: 7.9341e-04 - val_loss: 5.2086e-04
Epoch 6/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 53s 44ms/step - loss: 7.3319e-04 - val_loss: 5.1332e-04
Epoch 7/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 83s 44ms/step - loss: 6.8210e-04 - val_loss: 4.9652e-04
Epoch 8/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 82s 45ms/step - loss: 6.4266e-04 - val_loss: 5.0654e-04
Epoch 9/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 53s 44ms/step - loss: 6.1051e-04 - val_loss: 4.9820e-04
Epoch 10/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 55s 45ms/step - loss: 5.8873e-04 - val_loss: 4.8455e-04
Durchs

In [ ]:
# Liste der Optimizer und Lernraten
optimizers = {
    'Adam': Adam,
    'RMSprop': RMSprop,
    'Adadelta': Adadelta,
    'Adagrad': Adagrad
}

learning_rates = [0.001, 0.005, 0.01, 0.05, 0.1]

# Ergebnisse speichern
results = []

for opt_name, opt_func in optimizers.items():
    for lr in learning_rates:
        print(f"Testen mit Optimizer: {opt_name}, Lernrate: {lr}")

        # Modell erstellen
        model = Sequential()
        model.add(LSTM(units=64,
                       input_shape=(X_tr.shape[1], X_tr.shape[2]),
                       return_sequences=True))
        model.add(LSTM(units=64))
        model.add(Dense(1, activation='linear'))

        # Optimizer Instanziieren
        optimizer = opt_func(learning_rate=lr)

        # Modell kompilieren
        model.compile(optimizer=optimizer,
                      loss='mean_squared_error')

        # Training durchführen
        history = model.fit(X_tr, y_tr, epochs=10, batch_size=64,
                            validation_data=(X_v, y_v))

        # Validation Loss speichern
        val_loss = np.min(history.history['val_loss'])
        print(f"Validierungsverlust: {val_loss:.6f}")
        results.append({
            'optimizer': opt_name,
            'learning_rate': ,
            'val_loss': val_loss
        })

# Ergebnisse anzeigen
for res in results:
    print(f"Optimizer: {res['optimizer']}, Lernrate: {res['learning_rate']}, "
          f"Bester Validierungsverlust: {res['val_loss']:.6f}")

Testen mit Optimizer: Adam, Lernrate: 0.001


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 58s 45ms/step - loss: 0.0013 - val_loss: 4.3432e-04
Epoch 2/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 81s 45ms/step - loss: 3.7223e-04 - val_loss: 5.0666e-04
Epoch 3/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 81s 44ms/step - loss: 3.1966e-04 - val_loss: 3.2692e-04
Epoch 4/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 52s 43ms/step - loss: 2.9506e-04 - val_loss: 4.7283e-04
Epoch 5/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 83s 44ms/step - loss: 2.6849e-04 - val_loss: 3.2664e-04
Epoch 6/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 82s 44ms/step - loss: 2.5472e-04 - val_loss: 3.6051e-04
Epoch 7/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 83s 44ms/step - loss: 2.4715e-04 - val_loss: 4.4739e-04
Epoch 8/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 52s 43ms/step - loss: 2.5161e-04 - val_loss: 4.4212e-04
Epoch 9/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 83s 44ms/step - loss: 2.5884e-04 - val_loss: 2.9317e-04
Epoch 10/10
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 82s 44ms/step - loss: 2.4000e-04 - val_loss: 2.8536e-04
Validierungsverlust: 0.

In [ ]:
# Modell erstellen
model = Sequential()
model.add(LSTM(units=64,
                input_shape=(X_tr.shape[1], X_tr.shape[2]), dropout=0.2,
                return_sequences=True))
model.add(LSTM(units=64, dropout=0.2, return_sequences=True))
model.add(LSTM(units=64, dropout=0.2))
model.add(Dense(1, activation='linear'))

# Modell kompilieren
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='mean_squared_error')

# Training durchführen
history = model.fit(X_tr, y_tr, epochs=30, batch_size=64,
                    validation_data=(X_v, y_v))

# Validation Loss speichern
val_loss = np.min(history.history['val_loss'])
print(f"Validierungsverlust: {val_loss:.6f}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 86s 65ms/step - loss: 0.0014 - val_loss: 4.9358e-04
Epoch 2/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 77s 63ms/step - loss: 4.0058e-04 - val_loss: 4.6758e-04
Epoch 3/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 81s 63ms/step - loss: 3.5806e-04 - val_loss: 4.4847e-04
Epoch 4/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 83s 64ms/step - loss: 3.4129e-04 - val_loss: 4.4165e-04
Epoch 5/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 82s 64ms/step - loss: 3.0423e-04 - val_loss: 4.2664e-04
Epoch 6/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 86s 68ms/step - loss: 2.9368e-04 - val_loss: 3.9632e-04
Epoch 7/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 138s 65ms/step - loss: 3.1430e-04 - val_loss: 3.8609e-04
Epoch 8/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 83s 66ms/step - loss: 2.9196e-04 - val_loss: 3.4687e-04
Epoch 9/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 82s 66ms/step - loss: 2.7510e-04 - val_loss: 3.5435e-04
Epoch 10/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 80s 65ms/step - loss: 2.8062e-04 - val_loss: 4.0042e-04
Epoch 11/30
1219/1219 

In [ ]:
# Modell erstellen
model = Sequential()
model.add(LSTM(units=64,
                input_shape=(X_tr.shape[1], X_tr.shape[2]),
                return_sequences=True))
model.add(LSTM(units=64, return_sequences=True))
model.add(LSTM(units=64))
model.add(Dense(1, activation='linear'))

# Modell kompilieren
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='mean_squared_error')

# EarlyStopping Callback definieren
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Training durchführen
history = model.fit(X_tr, y_tr, epochs=30, batch_size=64,
                    validation_data=(X_v, y_v), callbacks=[early_stop])

# Validation Loss speichern
val_loss = np.min(history.history['val_loss'])
print(f"Validierungsverlust: {val_loss:.6f}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 103s 79ms/step - loss: 0.0014 - val_loss: 4.3069e-04
Epoch 2/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 138s 76ms/step - loss: 3.4718e-04 - val_loss: 4.0405e-04
Epoch 3/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 90s 74ms/step - loss: 3.2549e-04 - val_loss: 3.7297e-04
Epoch 4/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 143s 75ms/step - loss: 2.7192e-04 - val_loss: 3.4445e-04
Epoch 5/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 94s 77ms/step - loss: 2.6254e-04 - val_loss: 4.2253e-04
Epoch 6/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 140s 75ms/step - loss: 2.3764e-04 - val_loss: 3.1509e-04
Epoch 7/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 138s 72ms/step - loss: 2.5292e-04 - val_loss: 2.9791e-04
Epoch 8/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 145s 74ms/step - loss: 2.2201e-04 - val_loss: 3.3913e-04
Epoch 9/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 140s 73ms/step - loss: 2.3871e-04 - val_loss: 2.9185e-04
Epoch 10/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 90s 73ms/step - loss: 2.2302e-04 - val_loss: 3.0824e-04
Epoch 11/30
1219

In [ ]:
# Modell erstellen
model = Sequential()
model.add(LSTM(units=64,
                input_shape=(X_tr.shape[1], X_tr.shape[2]), dropout=0.2,
                return_sequences=True))
model.add(LSTM(units=64, dropout=0.2, return_sequences=True))
model.add(LSTM(units=64))
model.add(Dense(1, activation='linear'))

# Modell kompilieren
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='mean_squared_error')

# EarlyStopping Callback definieren
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Training durchführen
history = model.fit(X_tr, y_tr, epochs=30, batch_size=64,
                    validation_data=(X_v, y_v), callbacks=[early_stop])

# Validation Loss speichern
val_loss = np.min(history.history['val_loss'])
print(f"Validierungsverlust: {val_loss:.6f}")

Epoch 1/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 101s 77ms/step - loss: 8.3371e-04 - val_loss: 4.5621e-04
Epoch 2/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 139s 75ms/step - loss: 3.6987e-04 - val_loss: 4.3576e-04
Epoch 3/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 95s 78ms/step - loss: 3.4593e-04 - val_loss: 4.0431e-04
Epoch 4/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 139s 75ms/step - loss: 3.0854e-04 - val_loss: 3.3829e-04
Epoch 5/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 141s 75ms/step - loss: 2.8076e-04 - val_loss: 3.2526e-04
Epoch 6/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 92s 75ms/step - loss: 2.9379e-04 - val_loss: 3.8216e-04
Epoch 7/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 142s 75ms/step - loss: 2.8015e-04 - val_loss: 3.7360e-04
Epoch 8/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 145s 78ms/step - loss: 2.7679e-04 - val_loss: 3.3288e-04
Epoch 9/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 91s 75ms/step - loss: 2.9473e-04 - val_loss: 3.3608e-04
Epoch 10/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 147s 79ms/step - loss: 2.6520e-04 - val_loss: 3.6336e-04
Validierungs

In [ ]:
# Modell erstellen
model = Sequential()
model.add(LSTM(units=64,
                input_shape=(X_tr.shape[1], X_tr.shape[2]),
                kernel_regularizer=tf.keras.regularizers.l2(0.001),
                return_sequences=True))
model.add(LSTM(units=64, kernel_regularizer=tf.keras.regularizers.l2(0.001), return_sequences=True))
model.add(LSTM(units=64, kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(Dense(1, activation='linear'))

# Modell kompilieren
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='mean_squared_error')

# EarlyStopping Callback definieren
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Training durchführen
history = model.fit(X_tr, y_tr, epochs=30, batch_size=64,
                    validation_data=(X_v, y_v), callbacks=[early_stop])

# Validation Loss speichern
val_loss = np.min(history.history['val_loss'])
print(f"Validierungsverlust: {val_loss:.6f}")

Epoch 1/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 98s 74ms/step - loss: 0.0623 - val_loss: 0.0014
Epoch 2/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 144s 76ms/step - loss: 8.4519e-04 - val_loss: 5.2272e-04
Epoch 3/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 140s 74ms/step - loss: 3.9132e-04 - val_loss: 4.8209e-04
Epoch 4/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 142s 74ms/step - loss: 3.8678e-04 - val_loss: 5.7125e-04
Epoch 5/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 90s 74ms/step - loss: 3.7539e-04 - val_loss: 4.9188e-04
Epoch 6/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 139s 72ms/step - loss: 4.0766e-04 - val_loss: 4.8395e-04
Epoch 7/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 146s 75ms/step - loss: 3.9646e-04 - val_loss: 5.0733e-04
Epoch 8/30
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 88s 72ms/step - loss: 3.9305e-04 - val_loss: 4.8556e-04
Validierungsverlust: 0.000482


In [ ]:
# Modell erstellen
model = Sequential()
model.add(LSTM(units=64,
                input_shape=(X_tr.shape[1], X_tr.shape[2]),
                return_sequences=True))
model.add(LSTM(units=64, return_sequences=True))
model.add(LSTM(units=64))
model.add(Dense(1, activation='linear'))

# Modell kompilieren
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='mean_squared_error')

# EarlyStopping Callback definieren
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Lernratenabbaufaktor
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-6)

# Training durchführen
history = model.fit(X_tr, y_tr, epochs=50, batch_size=64,
                    validation_data=(X_v, y_v), callbacks=[early_stop, reduce_lr])

# Validation Loss speichern
val_loss = np.min(history.history['val_loss'])
print(f"Validierungsverlust: {val_loss:.6f}")

Epoch 1/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 98s 75ms/step - loss: 0.0042 - val_loss: 4.5078e-04 - learning_rate: 1.0000e-04
Epoch 2/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 141s 74ms/step - loss: 3.6349e-04 - val_loss: 4.4479e-04 - learning_rate: 1.0000e-04
Epoch 3/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 96s 79ms/step - loss: 3.3521e-04 - val_loss: 4.5937e-04 - learning_rate: 1.0000e-04
Epoch 4/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 141s 78ms/step - loss: 3.4140e-04 - val_loss: 4.1754e-04 - learning_rate: 1.0000e-04
Epoch 5/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 91s 75ms/step - loss: 3.1700e-04 - val_loss: 3.9706e-04 - learning_rate: 1.0000e-05
Epoch 6/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 140s 73ms/step - loss: 3.0010e-04 - val_loss: 3.9473e-04 - learning_rate: 1.0000e-05
Epoch 7/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 142s 73ms/step - loss: 3.0073e-04 - val_loss: 4.0010e-04 - learning_rate: 1.0000e-05
Epoch 8/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 142s 73ms/step - loss: 2.8572e-04 - val_loss: 3.9642e-04 - learning_rate: 1.

In [ ]:
# Modell erstellen
model_LSTM_001 = Sequential()
model_LSTM_001.add(LSTM(units=64,
                input_shape=(X_tr.shape[1], X_tr.shape[2]),
                return_sequences=True))
model_LSTM_001.add(LSTM(units=64, return_sequences=True))
model_LSTM_001.add(LSTM(units=64))
model_LSTM_001.add(Dense(1, activation='linear'))

# Modell kompilieren
model_LSTM_001.compile(optimizer=Adam(learning_rate=0.001),
              loss='mean_squared_error')

# EarlyStopping Callback definieren
early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

# Lernratenabbaufaktor
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-8)

# Training durchführen
history = model_LSTM_001.fit(X_tr, y_tr, epochs=50, batch_size=64,
                    validation_data=(X_v, y_v), callbacks=[early_stop, reduce_lr])

# Validation Loss speichern
val_loss = np.min(history.history['val_loss'])
print(f"Validierungsverlust: {val_loss:.6f}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 102s 79ms/step - loss: 0.0021 - val_loss: 4.2933e-04 - learning_rate: 0.0010
Epoch 2/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 85s 70ms/step - loss: 3.4405e-04 - val_loss: 4.7418e-04 - learning_rate: 0.0010
Epoch 3/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 140s 68ms/step - loss: 3.4235e-04 - val_loss: 4.6198e-04 - learning_rate: 0.0010
Epoch 4/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 84s 69ms/step - loss: 2.7440e-04 - val_loss: 3.5890e-04 - learning_rate: 0.0010
Epoch 5/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 84s 69ms/step - loss: 2.4203e-04 - val_loss: 3.1504e-04 - learning_rate: 1.0000e-04
Epoch 6/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 84s 69ms/step - loss: 2.3137e-04 - val_loss: 3.1187e-04 - learning_rate: 1.0000e-04
Epoch 7/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 142s 69ms/step - loss: 2.2438e-04 - val_loss: 3.2231e-04 - learning_rate: 1.0000e-04
Epoch 8/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 142s 69ms/step - loss: 2.2987e-04 - val_loss: 3.1280e-04 - learning_rate: 1.0000e-04
Epoch 9/

In [ ]:
# Modell erstellen
model_SGD = Sequential()
model_SGD.add(LSTM(units=64,
                input_shape=(X_tr.shape[1], X_tr.shape[2]),
                return_sequences=True))
model_SGD.add(LSTM(units=64, return_sequences=True))
model_SGD.add(LSTM(units=64))
model_SGD.add(Dense(1, activation='linear'))

# Modell kompilieren
model_SGD.compile(optimizer=SGD(learning_rate=0.001),
              loss='mean_squared_error')

# EarlyStopping Callback definieren
early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

# Lernratenabbaufaktor
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-8)

# Training durchführen
history = model_SGD.fit(X_tr, y_tr, epochs=50, batch_size=64,
                    validation_data=(X_v, y_v), callbacks=[early_stop, reduce_lr])

# Validation Loss speichern
val_loss = np.min(history.history['val_loss'])
print(f"Validierungsverlust: {val_loss:.6f}")

Epoch 1/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 122s 78ms/step - loss: 0.0070 - val_loss: 0.0013 - learning_rate: 0.0010
Epoch 2/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 95s 78ms/step - loss: 0.0011 - val_loss: 8.4081e-04 - learning_rate: 0.0010
Epoch 3/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 146s 81ms/step - loss: 7.4083e-04 - val_loss: 7.1098e-04 - learning_rate: 0.0010
Epoch 4/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 101s 83ms/step - loss: 5.9844e-04 - val_loss: 6.5733e-04 - learning_rate: 0.0010
Epoch 5/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 92s 75ms/step - loss: 5.4517e-04 - val_loss: 6.1751e-04 - learning_rate: 0.0010
Epoch 6/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 148s 80ms/step - loss: 4.9765e-04 - val_loss: 5.9261e-04 - learning_rate: 0.0010
Epoch 7/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 94s 77ms/step - loss: 4.7758e-04 - val_loss: 5.7564e-04 - learning_rate: 0.0010
Epoch 8/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 91s 75ms/step - loss: 4.6285e-04 - val_loss: 5.6453e-04 - learning_rate: 0.0010
Epoch 9/50
1219/1219 ━━━━━━━━━━━

In [ ]:
# Modell erstellen
model_LSTM_DO = Sequential()
model_LSTM_DO.add(LSTM(units=64,
                input_shape=(X_tr.shape[1], X_tr.shape[2]),
                dropout = 0.2,
                return_sequences=True))
model_LSTM_DO.add(LSTM(units=64, dropout = 0.2, return_sequences=True))
model_LSTM_DO.add(LSTM(units=64, dropout = 0.2))
model_LSTM_DO.add(Dense(1, activation='linear'))

# Modell kompilieren
model_LSTM_DO.compile(optimizer=Adam(learning_rate=0.001),
              loss='mean_squared_error')

# EarlyStopping Callback definieren
early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

# Lernratenabbaufaktor
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-8)

# Training durchführen
history = model_LSTM_DO.fit(X_tr, y_tr, epochs=50, batch_size=64,
                    validation_data=(X_v, y_v), callbacks=[early_stop, reduce_lr])

# Validation Loss speichern
val_loss = np.min(history.history['val_loss'])
print(f"Validierungsverlust: {val_loss:.6f}")

Epoch 1/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 92s 71ms/step - loss: 0.0014 - val_loss: 5.0819e-04 - learning_rate: 0.0010
Epoch 2/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 141s 70ms/step - loss: 3.7957e-04 - val_loss: 4.7295e-04 - learning_rate: 0.0010
Epoch 3/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 143s 71ms/step - loss: 3.5508e-04 - val_loss: 3.9411e-04 - learning_rate: 0.0010
Epoch 4/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 141s 70ms/step - loss: 3.1143e-04 - val_loss: 3.5414e-04 - learning_rate: 0.0010
Epoch 5/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 142s 70ms/step - loss: 2.9931e-04 - val_loss: 3.4003e-04 - learning_rate: 0.0010
Epoch 6/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 141s 70ms/step - loss: 2.8613e-04 - val_loss: 3.3823e-04 - learning_rate: 0.0010
Epoch 7/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 143s 71ms/step - loss: 2.7399e-04 - val_loss: 3.4422e-04 - learning_rate: 1.0000e-04
Epoch 8/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 141s 70ms/step - loss: 2.6984e-04 - val_loss: 3.5739e-04 - learning_rate: 1.0000e-04
Epoch 9/50
12

In [ ]:
# Modell erstellen
model_LSTM_0001 = Sequential()
model_LSTM_0001.add(LSTM(units=64,
                input_shape=(X_tr.shape[1], X_tr.shape[2]),
                return_sequences=True))
model_LSTM_0001.add(LSTM(units=64, return_sequences=True))
model_LSTM_0001.add(LSTM(units=64))
model_LSTM_0001.add(Dense(1, activation='linear'))

# Modell kompilieren
model_LSTM_0001.compile(optimizer=Adam(learning_rate=0.0001),
              loss='mean_squared_error')

# EarlyStopping Callback definieren
early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

# Lernratenabbaufaktor
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-9)

# Training durchführen
history = model_LSTM_0001.fit(X_tr, y_tr, epochs=50, batch_size=64,
                    validation_data=(X_v, y_v), callbacks=[early_stop, reduce_lr])

# Validation Loss speichern
val_loss = np.min(history.history['val_loss'])
print(f"Validierungsverlust: {val_loss:.6f}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 77s 58ms/step - loss: 8.8026e-04 - val_loss: 4.7186e-04 - learning_rate: 1.0000e-04
Epoch 2/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 70s 58ms/step - loss: 3.5326e-04 - val_loss: 4.3609e-04 - learning_rate: 1.0000e-04
Epoch 3/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 82s 58ms/step - loss: 3.5641e-04 - val_loss: 4.4050e-04 - learning_rate: 1.0000e-04
Epoch 4/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 82s 58ms/step - loss: 3.1904e-04 - val_loss: 5.6112e-04 - learning_rate: 1.0000e-04
Epoch 5/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 81s 58ms/step - loss: 3.3128e-04 - val_loss: 4.1220e-04 - learning_rate: 1.0000e-05
Epoch 6/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 71s 58ms/step - loss: 3.0156e-04 - val_loss: 4.1422e-04 - learning_rate: 1.0000e-05
Epoch 7/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 82s 58ms/step - loss: 3.0584e-04 - val_loss: 4.0689e-04 - learning_rate: 1.0000e-05
Epoch 8/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 82s 58ms/step - loss: 3.1208e-04 - val_loss: 4.0810e-04 - learning_rate: 1.0

In [ ]:
# Modell erstellen
model_LSTM_0001_5 = Sequential()
model_LSTM_0001_5.add(LSTM(units=64,
                input_shape=(X_tr.shape[1], X_tr.shape[2]),
                return_sequences=True))
model_LSTM_0001_5.add(LSTM(units=64, return_sequences=True))
model_LSTM_0001_5.add(LSTM(units=64))
model_LSTM_0001_5.add(Dense(1, activation='linear'))

# Modell kompilieren
model_LSTM_0001_5.compile(optimizer=Adam(learning_rate=0.0001),
              loss='mean_squared_error')

# EarlyStopping Callback definieren
early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

# Lernratenabbaufaktor
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-9)

# Training durchführen
history = model_LSTM_0001_5.fit(X_tr, y_tr, epochs=50, batch_size=64,
                    validation_data=(X_v, y_v), callbacks=[early_stop, reduce_lr])

# Validation Loss speichern
val_loss = np.min(history.history['val_loss'])
print(f"Validierungsverlust: {val_loss:.6f}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 91s 70ms/step - loss: 8.4203e-04 - val_loss: 4.5039e-04 - learning_rate: 1.0000e-04
Epoch 2/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 136s 65ms/step - loss: 3.4639e-04 - val_loss: 4.1966e-04 - learning_rate: 1.0000e-04
Epoch 3/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 75s 59ms/step - loss: 3.1876e-04 - val_loss: 3.9931e-04 - learning_rate: 1.0000e-04
Epoch 4/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 82s 59ms/step - loss: 3.2765e-04 - val_loss: 4.0701e-04 - learning_rate: 1.0000e-04
Epoch 5/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 83s 60ms/step - loss: 2.8696e-04 - val_loss: 3.7338e-04 - learning_rate: 5.0000e-05
Epoch 6/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 72s 59ms/step - loss: 2.7565e-04 - val_loss: 3.5961e-04 - learning_rate: 5.0000e-05
Epoch 7/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 85s 62ms/step - loss: 2.7570e-04 - val_loss: 3.6153e-04 - learning_rate: 5.0000e-05
Epoch 8/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 79s 59ms/step - loss: 2.7405e-04 - val_loss: 3.5282e-04 - learning_rate: 2.

In [ ]:
# Modell erstellen
model_LSTM_001_5 = Sequential()
model_LSTM_001_5.add(LSTM(units=64,
                input_shape=(X_tr.shape[1], X_tr.shape[2]),
                return_sequences=True))
model_LSTM_001_5.add(LSTM(units=64, return_sequences=True))
model_LSTM_001_5.add(LSTM(units=64))
model_LSTM_001_5.add(Dense(1, activation='linear'))

# Modell kompilieren
model_LSTM_001_5.compile(optimizer=Adam(learning_rate=0.001),
              loss='mean_squared_error')

# EarlyStopping Callback definieren
early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

# Lernratenabbaufaktor
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-8)

# Training durchführen
history = model_LSTM_001_5.fit(X_tr, y_tr, epochs=50, batch_size=64,
                    validation_data=(X_v, y_v), callbacks=[early_stop, reduce_lr])

# Validation Loss speichern
val_loss = np.min(history.history['val_loss'])
print(f"Validierungsverlust: {val_loss:.6f}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 99s 76ms/step - loss: 9.6344e-04 - val_loss: 4.3506e-04 - learning_rate: 0.0010
Epoch 2/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 91s 75ms/step - loss: 3.1963e-04 - val_loss: 3.5940e-04 - learning_rate: 0.0010
Epoch 3/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 141s 74ms/step - loss: 2.7694e-04 - val_loss: 3.1715e-04 - learning_rate: 0.0010
Epoch 4/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 143s 75ms/step - loss: 2.5291e-04 - val_loss: 3.2420e-04 - learning_rate: 0.0010
Epoch 5/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 91s 75ms/step - loss: 2.3946e-04 - val_loss: 2.9644e-04 - learning_rate: 0.0010
Epoch 6/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 91s 75ms/step - loss: 2.6963e-04 - val_loss: 3.9705e-04 - learning_rate: 0.0010
Epoch 7/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 141s 74ms/step - loss: 2.2986e-04 - val_loss: 3.1599e-04 - learning_rate: 5.0000e-04
Epoch 8/50
1219/1219 ━━━━━━━━━━━━━━━━━━━━ 141s 74ms/step - loss: 2.3876e-04 - val_loss: 2.8714e-04 - learning_rate: 5.0000e-04
Epoch 9/50
1